The primary "trained model" in this Spotify music recommendation code is the **`song_cluster_pipeline`**, a scikit-learn `Pipeline` object consisting of:

- A `StandardScaler` (fitted on the numeric audio features).
- A `KMeans` model (with 20 clusters, trained on the scaled song data).

This pipeline handles the scaling and is used in the `recommend_songs()` function to transform the mean vector of input songs and the full dataset for computing cosine distances.

To export (save) this trained pipeline for later use (e.g., in a production app or another script), use **`joblib`** (recommended by scikit-learn for models with large NumPy arrays, as it's more efficient than pickle).

### Step 1: Import joblib (if not already imported)
Add this at the top with your other imports:
```python
import joblib
```

### Step 2: Save the trained pipeline after fitting
Add this code right after the block where `song_cluster_pipeline` is fitted:
```python
# Save the trained song clustering pipeline
joblib.dump(song_cluster_pipeline, 'spotify_song_cluster_pipeline.joblib')
```

This creates a file named `spotify_song_cluster_pipeline.joblib` in your current working directory.

### Step 3: (Optional) Save the dataset for metadata
The recommendations pull song names, years, and artists from the `data` DataFrame. To make the recommender fully portable, save the dataset too:
```python
data.to_pickle('spotify_data.pkl')  # Or use joblib: joblib.dump(data, 'spotify_data.joblib')
```

### How to Load and Reuse the Model Later
In a new script or app:
```python
import joblib
import pandas as pd
from scipy.spatial.distance import cdist

# Load the pipeline and data
song_cluster_pipeline = joblib.load('spotify_song_cluster_pipeline.joblib')
data = pd.read_pickle('spotify_data.pkl')  # Or joblib.load()

number_cols = ['valence', 'year', 'acousticness', 'danceability', 'duration_ms', 'energy', 'explicit',
               'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'popularity', 'speechiness', 'tempo']

# Reuse the same functions (get_mean_vector, recommend_songs, etc.)
# ... paste or import the function definitions ...

# Example recommendation
recommend_songs([{'name': 'Come As You Are', 'year': 1991}, ...], data)
```

### Notes
- **Why joblib?** It's faster and uses less disk space for scikit-learn objects compared to Python's built-in `pickle`.
- Alternative with pickle:
  ```python
  import pickle
  with open('spotify_song_cluster_pipeline.pkl', 'wb') as f:
      pickle.dump(song_cluster_pipeline, f)
  ```
- The genre clustering pipeline or PCA/t-SNE pipelines are only for visualization and not needed for recommendations.
- If deploying (e.g., Flask app), load the model once at startup for efficiency.
- The Spotipy part (for fetching missing songs) remains separate and requires Spotify API credentials.

This allows you to reuse the exact trained scaling + clustering without retraining on the large dataset.